# 🚨 Agricultural Alert System & Forecasting

This notebook implements rule-based and ML-driven alert systems for agricultural monitoring.

## Features:
- Rule-based alerts (Fungal risk, Pest risk, Soil temperature, Rainfall anomaly)
- ML-driven alert prediction
- GPS-based weather forecasting
- 7-day forecast alerts
- Comprehensive alert dashboard

In [1]:
# Import required libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Date and time handling
from datetime import datetime, timedelta
import pytz

# Machine Learning
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Utils
import json
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm

# Set style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)
sns.set_palette("husl")

print("📚 Libraries imported successfully!")
print(f"Current time: {datetime.now()}")

📚 Libraries imported successfully!
Current time: 2025-07-31 11:14:07.442807


In [2]:
# Configuration
FARM_LOCATION = (41.8781, -93.0977)  # Des Moines, Iowa (example farm)
CROP_TYPE = "corn"
FORECAST_DAYS = 7

# Paths
BASE_PATH = Path("/Users/debabratapattnayak/web-dev/greencast")
RESULTS_PATH = BASE_PATH / "ml_models" / "results"
ALERT_DATA_PATH = RESULTS_PATH / "alerts"

# Create directories
RESULTS_PATH.mkdir(parents=True, exist_ok=True)
ALERT_DATA_PATH.mkdir(parents=True, exist_ok=True)

print(f"🌾 Farm Location: {FARM_LOCATION} (Lat, Lon)")
print(f"🌱 Crop Type: {CROP_TYPE}")
print(f"📁 Results will be saved to: {RESULTS_PATH}")

🌾 Farm Location: (41.8781, -93.0977) (Lat, Lon)
🌱 Crop Type: corn
📁 Results will be saved to: /Users/debabratapattnayak/web-dev/greencast/ml_models/results


## 🌤️ Weather Data Simulation

Since we don't have a real weather API key, we'll simulate realistic weather data based on GPS coordinates.

In [3]:
class WeatherSimulator:
    """Simulate realistic weather data for GPS coordinates"""
    
    def __init__(self, lat: float, lon: float):
        self.lat = lat
        self.lon = lon
        np.random.seed(int(lat * lon * 1000) % 1000)
    
    def get_current_weather(self) -> Dict:
        """Get current weather data"""
        
        current_weather = {
            'temperature': np.random.normal(25, 8),
            'humidity': np.random.normal(65, 15),
            'pressure': np.random.normal(1013, 20),
            'wind_speed': np.random.normal(5, 3),
            'rainfall': max(0, np.random.normal(2, 5)),
            'soil_temp': np.random.normal(22, 6),
            'uv_index': np.random.uniform(0, 11),
            'timestamp': datetime.now().isoformat()
        }
        
        # Ensure realistic ranges
        current_weather['humidity'] = np.clip(current_weather['humidity'], 30, 95)
        current_weather['wind_speed'] = max(0, current_weather['wind_speed'])
        current_weather['soil_temp'] = np.clip(current_weather['soil_temp'], 5, 40)
        
        return current_weather
    
    def get_7day_forecast(self) -> List[Dict]:
        """Get 7-day weather forecast"""
        
        forecast = []
        
        for day in range(7):
            date = datetime.now() + timedelta(days=day)
            
            # Simulate seasonal trends
            base_temp = 25 + 5 * np.sin(date.timetuple().tm_yday * 2 * np.pi / 365)
            
            day_weather = {
                'date': date.strftime('%Y-%m-%d'),
                'temperature_max': base_temp + np.random.normal(5, 3),
                'temperature_min': base_temp + np.random.normal(-5, 3),
                'temperature_avg': base_temp + np.random.normal(0, 2),
                'humidity': np.random.normal(65, 15),
                'rainfall': max(0, np.random.normal(3, 8)),
                'wind_speed': np.random.normal(5, 3),
                'soil_temp': base_temp + np.random.normal(-3, 2),
                'pressure': np.random.normal(1013, 15),
                'uv_index': np.random.uniform(2, 10)
            }
            
            # Ensure realistic ranges
            day_weather['humidity'] = np.clip(day_weather['humidity'], 30, 95)
            day_weather['wind_speed'] = max(0, day_weather['wind_speed'])
            day_weather['soil_temp'] = np.clip(day_weather['soil_temp'], 5, 40)
            
            forecast.append(day_weather)
        
        return forecast

# Initialize weather simulator
weather_sim = WeatherSimulator(*FARM_LOCATION)

# Get current weather
current_weather = weather_sim.get_current_weather()
print("🌤️ Current Weather Conditions:")
for key, value in current_weather.items():
    if key != 'timestamp':
        if isinstance(value, float):
            print(f"  {key.replace('_', ' ').title()}: {value:.1f}")
        else:
            print(f"  {key.replace('_', ' ').title()}: {value}")

# Get 7-day forecast
forecast = weather_sim.get_7day_forecast()
print(f"\n📅 7-Day Forecast Generated ({len(forecast)} days)")

# Display forecast summary
forecast_df = pd.DataFrame(forecast)
print("\n📊 Forecast Summary:")
forecast_df[['date', 'temperature_avg', 'humidity', 'rainfall']].head()

🌤️ Current Weather Conditions:
  Temperature: 32.0
  Humidity: 64.4
  Pressure: 1028.8
  Wind Speed: 10.1
  Rainfall: 0
  Soil Temp: 36.4
  Uv Index: 0.7

📅 7-Day Forecast Generated (7 days)

📊 Forecast Summary:


,date,temperature_avg,humidity,rainfall
0,2025-07-31,22.397692,69.059145,6.735268
1,2025-08-01,26.229902,80.201780,2.050959
2,2025-08-02,21.333802,40.199719,0.000000
3,2025-08-03,24.099469,81.695103,0.000000
4,2025-08-04,22.759036,66.000489,8.627003
